In [ ]:
#!pip install mne

In [3]:
import mne

In [6]:
raw=mne.io.read_raw_gdf('E:\Dataset\EEG\BCI IV\BCICIV_2a_gdf\A04T.gdf',
                         eog=['EOG-left', 'EOG-central', 'EOG-right'])
raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])

Extracting EDF parameters from E:\Dataset\EEG\BCI IV\BCICIV_2a_gdf\A04T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...


C:\Users\daidu\AppData\Local\Programs\Python\Python37\lib\contextlib.py:119: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


<RawGDF | A04T.gdf, 22 x 600915 (2403.7 s), ~26 kB, data not loaded>

In [7]:
raw.annotations

<Annotations | 610 segments: 1023 (26), 1072 (1), 32766 (7), 768 (288), ...>

In [8]:
events=mne.events_from_annotations(raw)
events[1]

Used Annotations descriptions: ['1023', '1072', '32766', '768', '769', '770', '771', '772']


{'1023': 1,
 '1072': 2,
 '32766': 3,
 '768': 4,
 '769': 5,
 '770': 6,
 '771': 7,
 '772': 8}

In [9]:
raw

<RawGDF | A04T.gdf, 22 x 600915 (2403.7 s), ~26 kB, data not loaded>

In [10]:
data = raw.get_data()

In [11]:
data.shape

(22, 600915)

In [2]:
import os
import pickle
import shutil

from subfunc.generate_artificial_data import generate_artificial_data
from subfunc.preprocessing import pca
from tcl.tcl_train import train


# Parameters ==================================================
# =============================================================

# Data generation ---------------------------------------------
random_seed = 0 # random seed
num_comp = 20 # number of components (dimension)
num_segment = 256 # number of segments
num_segmentdata = 512 # number of data-points in each segment
num_layer = 5 # number of layers of mixing-MLP

# MLP ---------------------------------------------------------
list_hidden_nodes = [40, 40, 40, 40, 20]
# list of the number of nodes of each hidden layer of feature-MLP
# [layer1, layer2, ..., layer(num_layer)]

# Training ----------------------------------------------------
initial_learning_rate = 0.01 # initial learning rate
momentum = 0.9 # momentum parameter of SGD
max_steps = int(7e5) # number of iterations (mini-batches)
decay_steps = int(5e5) # decay steps (tf.train.exponential_decay)
decay_factor = 0.1 # decay factor (tf.train.exponential_decay)
batch_size = 512 # mini-batch size
moving_average_decay = 0.999 # moving average decay of variables to be saved
checkpoint_steps = 1e5 # interval to save checkpoint

# for MLR initialization
max_steps_init = int(7e4) # number of iterations (mini-batches) for initializing only MLR
decay_steps_init = int(5e4) # decay steps for initializing only MLR

# Other -------------------------------------------------------
# # Note: save folder must be under ./storage
train_dir = './storage/temp' # save directory (Caution!! this folder will be removed at first)
saveparmpath = os.path.join(train_dir, 'parm.pkl') # file name to save parameters



# =============================================================
# =============================================================

# Prepare save folder -----------------------------------------
if train_dir.find("./storage/") > -1:
    if os.path.exists(train_dir):
        print("delete savefolder: {0:s}...".format(train_dir))
        shutil.rmtree(train_dir)  # Remove folder
    print("make savefolder: {0:s}...".format(train_dir))
    os.makedirs(train_dir)  # Make folder
else:
    assert False, "savefolder looks wrong"


# Generate sensor signal --------------------------------------
sensor, source, label = generate_artificial_data(num_comp=num_comp,
                                                 num_segment=num_segment,
                                                 num_segmentdata=num_segmentdata,
                                                 num_layer=num_layer,
                                                 random_seed=random_seed)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


make savefolder: ./storage/temp...
Generating source...
Generating sensor signal...
    cond thresh: 36.190674
    L4: cond=303.126314
    L4: cond=56.555460
    L4: cond=171.889794
    L4: cond=61.038865
    L4: cond=40.923516
    L4: cond=31.955147
    L3: cond=166.553445
    L3: cond=46.538361
    L3: cond=413.509699
    L3: cond=102.349694
    L3: cond=470.571844
    L3: cond=57.997376
    L3: cond=31.264658
    L2: cond=91.444432
    L2: cond=60.877266
    L2: cond=35.757592
    L1: cond=60.887093
    L1: cond=87.427231
    L1: cond=45.493049
    L1: cond=166.029907
    L1: cond=29.062968
    L0: cond=58.152693
    L0: con

In [ ]:
# Train model -------------------------------------------------
train(data,
      label,
      num_class = num_segment,
      list_hidden_nodes = list_hidden_nodes,
      initial_learning_rate = initial_learning_rate,
      momentum = momentum,
      max_steps = max_steps,
      decay_steps = decay_steps,
      decay_factor = decay_factor,
      batch_size = batch_size,
      train_dir = train_dir,
      checkpoint_steps = checkpoint_steps,
      moving_average_decay = moving_average_decay,
      load_file=init_model_path,
      random_seed = random_seed)


# Save parameters necessary for evaluation --------------------
model_parm = {'random_seed':random_seed,
              'num_comp':num_comp,
              'num_segment':num_segment,
              'num_segmentdata':num_segmentdata,
              'num_layer':num_layer,
              'list_hidden_nodes':list_hidden_nodes,
              'moving_average_decay':moving_average_decay,
              'pca_parm':pca_parm}

print("Save parameters...")
with open(saveparmpath, 'wb') as f:
    pickle.dump(model_parm, f, pickle.HIGHEST_PROTOCOL)

print("done.")